<div class="align-center">
<a href="https://oumi.ai/"><img src="https://oumi.ai/docs/en/latest/_static/logo/header_logo.png" height="200"></a>

[![Documentation](https://img.shields.io/badge/Documentation-latest-blue.svg)](https://oumi.ai/docs/en/latest/index.html)
[![Discord](https://img.shields.io/discord/1286348126797430814?label=Discord)](https://discord.gg/oumi)
[![GitHub Repo stars](https://img.shields.io/github/stars/oumi-ai/oumi)](https://github.com/oumi-ai/oumi)
</div>

👋 Welcome to Open Universal Machine Intelligence (Oumi)!

🚀 Oumi is a fully open-source platform that streamlines the entire lifecycle of foundation models - from [data preparation](https://oumi.ai/docs/en/latest/resources/datasets/datasets.html) and [training](hhttps://oumi.ai/docs/en/latest/user_guides/train/train.html) to [evaluation](https://oumi.ai/docs/en/latest/user_guides/evaluate/evaluate.html) and [deployment](https://oumi.ai/docs/en/latest/user_guides/launch/launch.html). Whether you're developing on a laptop, launching large scale experiments on a cluster, or deploying models in production, Oumi provides the tools and workflows you need.

🤝 Make sure to join our [Discord community](https://discord.gg/oumi) to get help, share your experiences, and contribute to the project! If you are interested in joining one of the community's open-science efforts, check out our [open collaboration](https://oumi.ai/community) page.

⭐ If you like Oumi and you would like to support it, please give it a star on [GitHub](https://github.com/oumi-ai/oumi).

# Distillation Overview

In this tutorial, we'll fine-tune a small language model (SLM) from the outputs of a large language model (LLM).

We'll use the Oumi framework to streamline the process and achieve high-quality results.

We'll cover the following topics:
1. Prerequisites
2. Data Preparation & Sanity Checks
3. Training Config Preparation
4. Launching Training
5. Monitoring Progress
6. Evaluation
7. Analyzing Results
8. Inference


# Prerequisites
## Hardware
The defaults in this tutorial are scaled down for demonstration purposes.

The true values are left to code comments within each section.

We recommend 8xA100-80GB GPUs to complete in a timely manner with adequate performance.

## Oumi Installation

First, let's install Oumi. You can find more detailed instructions [here](https://oumi.ai/docs/en/latest/get_started/installation.html).

If you have a GPU, you can run the following commands to install Oumi:


In [2]:
%pip install uv -q
!uv pip install oumi[gpu] --no-progress --system

Using Python 3.11.11 environment at: /usr
Resolved 147 packages in 1.82s
Uninstalled 13 packages in 583ms
Installed 13 packages in 230ms
 - nvidia-cublas-cu12==12.4.5.8
 + nvidia-cublas-cu12==12.1.3.1
 - nvidia-cuda-cupti-cu12==12.4.127
 + nvidia-cuda-cupti-cu12==12.1.105
 - nvidia-cuda-nvrtc-cu12==12.4.127
 + nvidia-cuda-nvrtc-cu12==12.1.105
 - nvidia-cuda-runtime-cu12==12.4.127
 + nvidia-cuda-runtime-cu12==12.1.105
 - nvidia-cufft-cu12==11.2.1.3
 + nvidia-cufft-cu12==11.0.2.54
 - nvidia-curand-cu12==10.3.5.147
 + nvidia-curand-cu12==10.3.2.106
 - nvidia-cusolver-cu12==11.6.1.9
 + nvidia-cusolver-cu12==11.4.5.107
 - nvidia-cusparse-cu12==12.3.1.170
 + nvidia-cusparse-cu12==12.1.0.106
 - nvidia-nccl-cu12==2.21.5
 + nvidia-nccl-cu12==2.20.5
 - nvidia-nvtx-cu12==12.4.127
 + nvidia-nvtx-cu12==12.1.105
 - torch==2.5.1
 + torch==2.4.1
 - torchvision==0.20.1
 + torchvision==0.19.1
 - triton==3.1.0
 + triton==3.0.0


In [4]:
!pip install vllm --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 193.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 166.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 237.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 220.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 228.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 185.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 203.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 227.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 201.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 214.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 222.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 315.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

**WARNING:** After the first `pip install`, you may have to restart the notebook for the package updates to take effect (Colab Menu: `Runtime` -> `Restart Session`)

## Creating our working directory
For our experiments, we'll use the following folder to save the model, training artifacts, and our working configs.

In [2]:
from pathlib import Path

tutorial_dir = "distillation_tutorial"

Path(tutorial_dir).mkdir(parents=True, exist_ok=True)

## Setup the environment

We'll need to set the following environment variables:
- [Optional] HF_TOKEN: Your [HuggingFace](https://huggingface.co/docs/hub/en/security-tokens) token, in case you want to access a private model.
- [Optional] WANDB_API_KEY: Your [wandb](https://wandb.ai) token, in case you want to log your experiments to wandb.

# Getting Started

## Model Download

For our purposes it will be much faster if we download our models first.

We'll use the `hf_transfer` package to download.

In [2]:
!pip install hf_transfer

In [3]:
!HF_HUB_ENABLE_HF_TRANSFER=1 \
    huggingface-cli download deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B \
    --exclude original/*

/root/.cache/huggingface/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/6393b7559e403fd1d80bfead361586fd6f630a4d


In [4]:
!HF_HUB_ENABLE_HF_TRANSFER=1 \
    huggingface-cli download deepseek-ai/DeepSeek-R1-Distill-Llama-8B \
    --exclude original/*

/root/.cache/huggingface/hub/models--deepseek-ai--DeepSeek-R1-Distill-Llama-8B/snapshots/ebf7e8d03db3d86a442d22d30d499abb7ec27bea


In [30]:
!HF_HUB_ENABLE_HF_TRANSFER=1 \
    huggingface-cli download unsloth/DeepSeek-R1-Distill-Qwen-1.5B-GGUF \
    --exclude original/*

.gitattributes: 100% 2.06k/2.06k [00:00<00:00, 13.6MB/s]
Download complete. Moving file to /root/.cache/huggingface/hub/models--unsloth--DeepSeek-R1-Distill-Qwen-1.5B-GGUF/blobs/a62d45a541d9ae81e9327e920e0ea6b5d293b6ba
DeepSeek-R1-Distill-Qwen-1.5B-Q2_K.gguf: 100% 753M/753M [00:07<00:00, 101MB/s]
Download complete. Moving file to /root/.cache/huggingface/hub/models--unsloth--DeepSeek-R1-Distill-Qwen-1.5B-GGUF/blobs/e18142b69b2dbdac59eca6bf77dde2054078003bcb9534e02e7ca1cf26eb5675
(…)eepSeek-R1-Distill-Qwen-1.5B-Q2_K_L.gguf: 100% 808M/808M [00:11<00:00, 71.8MB/s]
Download complete. Moving file to /root/.cache/huggingface/hub/models--unsloth--DeepSeek-R1-Distill-Qwen-1.5B-GGUF/blobs/6f6c9b97adba11427b13bc159a68881d13d99219165b3bff1b009b27018723db
(…)eepSeek-R1-Distill-Qwen-1.5B-Q3_K_M.gguf: 100% 924M/924M [00:05<00:00, 175MB/s]
Download complete. Moving file to /root/.cache/huggingface/hub/models--unsloth--DeepSeek-R1-Distill-Qwen-1.5B-GGUF/blobs/1bf6d8e3b6159186e0bf268d52bb22759a284cae31

## Baseline Evals

Before we can improve our small model, we should measure how well it performs on a benchmark compared to the larger model.

The below code will run the MMLU PRO Math task from LM Harness.

Note that this will take some time, so we've recorded our results below for your convenience:

| Model | MMLU Pro Math Accuracy |
|-------|------------------------|
| R1 Distill 1.5B | 38.49% +- 1.32% |
| R1 Distill 70B | 61.07% +- 1.33% |

### Run Evals

In [5]:
%%writefile $tutorial_dir/eval_small.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
  torch_dtype_str: "bfloat16"
  # shard_for_eval: True # Uncomment this line for multi-gpu setups.


tasks:
  - evaluation_platform: lm_harness
    task_name: mmlu_pro_math

output_dir: "distillation_tutorial/output/evaluation"
generation:
  batch_size: 1 # LM Harness recommends BS=1 for reproducibility.
  # batch_size: 256  # Replace with 256 for 8xA100-80GB

Overwriting distillation_tutorial/eval_small.yaml


In [8]:
!oumi evaluate -c "$tutorial_dir/eval_small.yaml"


@@@@@@@@@@@@@@@@@@@
@                 @
@   @@@@@  @  @   @
@   @   @  @  @   @
@   @@@@@  @@@@   @
@                 @
@   @@@@@@@   @   @
@   @  @  @   @   @
@   @  @  @   @   @
@                 @
@@@@@@@@@@@@@@@@@@@

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
2025-02-01 15:08:29.849362: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738422510.075148    5946 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738422510.138723    5946 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS w

In [6]:
%%writefile $tutorial_dir/eval_large.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
  torch_dtype_str: "bfloat16"
  shard_for_eval: True


tasks:
  - evaluation_platform: lm_harness
    task_name: mmlu_pro_math

output_dir: "distillation_tutorial/output/evaluation"
generation:
  batch_size: 1 # LM Harness recommends BS=1 for reproducibility.
  # batch_size: 64  # Replace with 64 for 8xA100-80GB

Overwriting distillation_tutorial/eval_large.yaml


In [10]:
!oumi evaluate -c "$tutorial_dir/eval_large.yaml"


@@@@@@@@@@@@@@@@@@@
@                 @
@   @@@@@  @  @   @
@   @   @  @  @   @
@   @@@@@  @@@@   @
@                 @
@   @@@@@@@   @   @
@   @  @  @   @   @
@   @  @  @   @   @
@                 @
@@@@@@@@@@@@@@@@@@@

2025-02-01 15:14:07.170006: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738422847.189999    7454 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738422847.196809    7454 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-01 15:14:07.218904: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following inst

## Prepare Inference Data

Now that we've set our baseline numbers, let's prepare the training data we'll use to improve 1.5B.

Given our goal is to improve MMLU Pro Math performance, we should ideally pick data that's similar.

`meta-math/MetaMathQA` is a good choice as it avoids test set contamination while being similar.

In [3]:
import os

import datasets
import torch

from oumi.core.configs import InferenceConfig
from oumi.core.types import Conversation, Message, Role
from oumi.inference import VLLMInferenceEngine

# This is needed for vLLM to use multiple GPUs in a notebook.
# If you're not running in a notebook, you can ignore this.
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

INFO 02-01 16:11:30 __init__.py:183] Automatically detected platform cuda.


In [4]:
dataset = datasets.load_dataset(
    "meta-math/MetaMathQA",
    revision="aa4f34d",
    split="train[:10000]",  # We'll focus only on the first 10k samples.
)

data = [sample["query"] for sample in dataset]
print(data[0])
print("num samples: ", len(data))

Gracie and Joe are choosing numbers on the complex plane. Joe chooses the point $1+2i$. Gracie chooses $-1+i$. How far apart are Gracie and Joe's points?
num samples:  10000


In [5]:
conversations = [
    Conversation(
        messages=[
            Message(role=Role.USER, content=prompt),
        ]
    )
    for prompt in data
]
print(conversations[0])

conversation_id=None messages=[USER: Gracie and Joe are choosing numbers on the complex plane. Joe chooses the point $1+2i$. Gracie chooses $-1+i$. How far apart are Gracie and Joe's points?] metadata={}


## Run Inference

Now that our data is in the right format for collecting responses, let's go ahead and run inference.

In [6]:
%%writefile $tutorial_dir/infer_large.yaml

model:
  #model_name: "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
  #model_name: "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
  model_name: "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-GGUF"
  torch_dtype_str: "bfloat16"
  model_max_length: 8192

generation:
  max_new_tokens: 8192

Overwriting distillation_tutorial/infer_large.yaml


In [45]:
!pip install --upgrade oumi[gpu]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [7]:
%%time

# Download, and load the model in memory
# This may take a while, depending on your internet speed.
# The inference engine only needs to be loaded once and can be
# reused for multiple conversations.
config_path = f"{tutorial_dir}/infer_large.yaml"
config = InferenceConfig.from_yaml(config_path)

from oumi.utils.transformers_utils import build_tokenizer

# Build the tokenizer with the specified model type
tokenizer = build_tokenizer(config.model, model_type="qwen") # Passing model_type here

inference_engine = VLLMInferenceEngine(
    config.model,
    #tensor_parallel_size=torch.cuda.device_count(),  # use all available GPUs
    tensor_parallel_size=1,
    # Enable prefix caching for vLLM.
    # This is key for performance when running prompts with a long prefix,
    # such as judging or conversations with large system prompts
    # or few-shot examples.
    enable_prefix_caching=True,
    # Use quantization to reduce memory footprint
    #quantization="gguf", # Add this line to enable quantization
    tokenizer=tokenizer # Pass the tokenizer to VLLMInferenceEngine
)

ModuleNotFoundError: No module named 'oumi.utils.transformers_utils'

In [ ]:
%%time

print(f"Running inference for {len(conversations)} conversations")

generations = inference_engine.infer(
    input=conversations,
    inference_config=config,
)
print(generations[0])

## Prepare Training Data

Now that we've finished collecting responses, let's go ahead and prepare the data for training and save it.

In [ ]:
conversation_dicts = [c.to_dict() for c in generations]
print(conversation_dicts[0])

In [ ]:
import pandas as pd

dataframe = pd.DataFrame(conversation_dicts)
print(dataframe)

In [ ]:
dataframe.to_json(f"{tutorial_dir}/math_train_10k.jsonl", orient="records", lines=True)

## Run Distillation

Now that the data is ready, we can begin distilling the model. For this form of distillation, we will be fully fine-tuning the model with supervised fine-tuning.

In [ ]:
%%writefile $tutorial_dir/train.yaml

model:
  model_name: "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
  trust_remote_code: true
  torch_dtype_str: "bfloat16"
  device_map: "auto"

data:
  train:
    datasets:
      - dataset_name: "text_sft_jsonl"
        dataset_path: "./distillation_tutorial/math_train_10k.jsonl"
        split: "train"
        shuffle: True
        seed: 42
    seed: 42

training:
  output_dir: "distillation_tutorial/output/finetune"

  # For a single GPU, the following gives us a batch size of 16
  # If training with multiple GPUs, feel free to reduce gradient_accumulation_steps
  per_device_train_batch_size: 2
  gradient_accumulation_steps: 1  # Reduce this to 1 for 8xA100-80GB GPUs

  # ***NOTE***
  # We set it to 10 steps to first verify that it works
  # Comment out the line below to have it train for 1 full epoch (all the data) instead.
  # Note: 1 full epoch will take about 13 minutes on 8xA100-80GB.
  max_steps: 10

  num_train_epochs: 1
  learning_rate: 1e-4
  warmup_ratio: 0.1
  logging_steps: 10
  save_steps: 0
  max_grad_norm: 10
  weight_decay: 0.01


  trainer_type: "TRL_SFT"
  optimizer: "adamw_torch_fused"
  enable_gradient_checkpointing: True
  gradient_checkpointing_kwargs:
    use_reentrant: False
  ddp_find_unused_parameters: False
  dataloader_num_workers: "auto"
  dataloader_prefetch_factor: 32
  empty_device_cache_steps: 1

### Single GPU

In [ ]:
!oumi train -c "$tutorial_dir/train.yaml"

### Multi-GPU

In [ ]:
!oumi distributed torchrun -m oumi train -c "$tutorial_dir/train.yaml"

## Evaluate

Now that we have a new distilled model, let's evaluate it on the same benchmark.

In [ ]:
%%writefile $tutorial_dir/eval_small_fft.yaml

model:
  model_name: "./distillation_tutorial/output/finetune/"
  torch_dtype_str: "bfloat16"
  shard_for_eval: True


tasks:
  - evaluation_platform: lm_harness
    task_name: mmlu_pro_math

output_dir: "distillation_tutorial/output/evaluation"
generation:
  batch_size: 1 # LM Harness recommends BS=1 for reproducibility.
  # batch_size: 256  # Replace with 256 for 8xA100-80GB

In [ ]:
!oumi evaluate -c "$tutorial_dir/eval_small_fft.yaml"

## Results

After we finetuned the model following the steps above, we achieved the following results:

| Model           | Accuracy        |
|-----------------|-----------------|
| R1 Distill 1.5B | 38.49% +- 1.32% |
| Oumi R1 Distill 1.5B | 42.41% +- 1.34% |
| R1 Distill 70B  | 61.07% +- 1.33% |